In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
from time import time
import json
import seaborn as sns
from numpy import asarray
from numpy import save
from numpy import load
import pickle
import joblib
import scipy
from sklearn.metrics import mean_squared_error

Set seed:

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)
sns.set_style("white")

In [ ]:
sns.set_style("white")

### Load Data

In [ ]:
with open ('../final_data/label_keys_ts.pkl', 'rb') as fp:
    label_keys = pickle.load(fp)
with open ('../final_data/feature_keys_ts.pkl', 'rb') as fp:
    feature_keys = pickle.load(fp)    

train_x = load('../final_data/train_x_norm.npy')
train_y = load('../final_data/train_y_norm.npy')
valid_x = load('../final_data/valid_x_norm.npy')
valid_y = load('../final_data/valid_y_norm.npy')
test_x = load('../final_data/test_x_norm.npy')
test_y = load('../final_data/test_y_norm.npy')
all_train_x = load('../final_data/all_train_x_norm.npy')
all_train_y = load('../final_data/all_train_y_norm.npy')

scaler_x = joblib.load("../final_data/scaler_x.save") 
scaler_y = joblib.load("../final_data/scaler_y.save") 

scaler_train_x = joblib.load("../final_data/scaler_train_x.save") 
scaler_train_y = joblib.load("../final_data/scaler_train_y.save")

train_unsc = pd.read_pickle("../final_data/train_unsc.pkl")
valid_unsc = pd.read_pickle("../final_data/valid_unsc.pkl")
test_unsc = pd.read_pickle("../final_data/test_unsc.pkl")
all_train_unsc = pd.read_pickle("../final_data/all_train_unsc.pkl")

train_norm = pd.read_pickle("../final_data/train_norm.pkl")
valid_norm = pd.read_pickle("../final_data/valid_norm.pkl")
test_norm = pd.read_pickle("../final_data/test_norm.pkl")
all_train_norm = pd.read_pickle("../final_data/all_train_norm.pkl")

cc4_data = pd.read_pickle("../final_data/cc4_data.pkl") 

In [ ]:
water_keys = ['WasserZ4FsInLproMin_integr',
 'WasserZ4LsInLproMin_integr',
 'WasserZ2bFsInLproMin_integr',
 'WasserZ2bLsInLproMin_integr',
 'WasserZ3bFsInLproMin_integr',
 'WasserZ1FsInLproMin_integr',
 'WasserZ3bLsInLproMin_integr',
 'WasserZ1LsInLproMin_integr',
 'WasserZ3aFsInLproMin_integr',
 'WasserZ3aLsInLproMin_integr',
 'WasserZ2aLsInLproMin_integr',
 'WasserZ2aFsInLproMin_integr',
 'WasserZ5LsInLproMin_integr',
 'WasserZ5FsInLproMin_integr',
 ]

feature_future_keys = [ele for ele in feature_keys if ele not in water_keys]

columns = cc4_data.columns.to_list()

### Load Model

In [ ]:
model = keras.models.load_model("../final_data/models/LSTMDropout_model")

## First Optimization

Set time point and steps in the future to optimize:

In [ ]:
seq_id = '474261_str_1' 
dt = 30
now = '2019-07-25 01:29:00+00:00'

### Looking at actual water for t+30:

In [ ]:
data = cc4_data[cc4_data['seq_id']==seq_id].reset_index(drop=True)
position = data[data['DATE_TIME']==now].index
real_water = data.loc[position[0]:position[0]+30][water_keys].reset_index(drop=True)

In [ ]:
fig, axes = plt.subplots( figsize=(20, 10),  sharex=True)
sns.barplot(data=real_water[sorted(water_keys)], color='dodgerblue')
plt.title('Real Water', fontsize=20, color='white') 

### Function for optimization:

Pseudocode for optimization, further explanation follows
    # def getData(now, dt):
                # data for LSTM
                # features_past (t...t-49):
                # features_future (except water) (t+1...t+30) 
                return features_past, features_future
 
    # def predict(W, features_past, features_future):
                # input = concat(features_past, concat(features_future, W))
                # for 50er-Slice in input :
                 T[t] = model.predict(50er-Slice)
                # return T
 
    # def cost(W, T_soll, features_past, features_future):
                # T = predict(W, features_past, features_future)
                # return sum(abs(T-T_soll))
                
    # def findBestWater(now, dt):
                # features_past*, features_future* = getData(now, dt)
                # W0 = mean of water for t...t-49
                # W = optimize.minimize(lambda W: cost(W,T_soll*, features_past*,      
                # features_future*),x0=W0)    
               #  return W

Function to get the data
- all attributes for the features for t...t-49:features_past
- all except the water attributes for the features for t+1...t+30:  features_future
- casting target temperature: T_soll
- scale all values between 0 and 1

In [ ]:
def getData(now, dt, cc4_data, seq_id):
    data = cc4_data[cc4_data['seq_id']==seq_id].reset_index(drop=True)
    position = data[data['DATE_TIME']==now].index
    
    features_past = data.loc[(position[0]-49):position[0]]  ##features for past
    features_past = features_past[feature_keys]
    features_past_norm = pd.DataFrame(scaler_x.transform(features_past), columns=feature_keys)
    features_future = data.loc[position[0]+1:position[0]+30]  ## features for future
    features_future = features_future[feature_keys]
    features_future_norm = pd.DataFrame(scaler_x.transform(features_future), columns=feature_keys)
    features_future_norm = features_future_norm[feature_future_keys]
    T_soll = np.array([cc4_data['ZielTempTreiberInC'].loc[position[0]:position[0]+30],
          cc4_data['ZielTempTreiberInC'].loc[position[0]:position[0]+30]]).transpose() ##casting target temperature
    T_soll_norm = scaler_y.transform(T_soll)
    return features_past_norm, features_future_norm, T_soll_norm

Function to predict temperature:
- combine optimized water W for future with features_future to features_future_new
- combinde features_future_new and features_past to input_arr: All the features to make predictions for considered time steps
- use model.predict to make prediction with model

In [ ]:
def predict(W, features_past, features_future):
    if isinstance(W, np.ndarray):
        W = pd.DataFrame(W.reshape((30,14)), columns=water_keys)
        
    features_future_new = pd.concat([features_future, W], axis=1).reindex(feature_keys, axis=1)
    input_arr = pd.concat([features_past,features_future_new], axis=0)
    features_slice = []
    feature_slice = [input_arr[i:i+50] for i in range(len(input_arr)-(50-1))]
    
    features= np.stack(feature_slice)
        
    T = model.predict(features)
    return T

objective function:
- mean absolute error (MAE) between predicted temperature (calculated with function to predict temperature) and casting target temperature

In [ ]:
def cost(W, T_soll, features_past, features_future):
    T = predict(W, features_past, features_future)
    return  np.mean(sum(abs(T-T_soll)))

Nelder-Mead optimizer:
- get all the data with get_data
- use average cooling water quantities for Initial guess W0
- use Nelder-Mead to optimize water quantities W for t+1...t+3, by minimizing objective function cost() (MAE between predicted and target temperature) where W is used.
- return best W

In [ ]:
def findBestWater(now, dt, cc4_data, strang_id):
    features_past, features_future, T_soll = getData(now, dt, cc4_data, seq_id)
    w = features_past[water_keys].mean().to_numpy().reshape((1,14)) ## water quantites fot t...t-4
    W0 = pd.DataFrame(np.repeat(w, repeats=dt, axis=0), columns=water_keys) ## initial guess
    W = scipy.optimize.minimize(lambda W: cost(W, T_soll, features_past, features_future) , x0=W0,
                                 options={ 'disp':True,  'adaptive':True, 'maxfev':200000},  ##optimize W by minimizing 
                                method = 'Nelder-Mead',                 ## MAE between T_pred and T_soll
                               )    
    return W

#### Either run optimization:

In [ ]:
feature_keys

In [ ]:
# np.random.seed(42)
# W_473289_str_1 = findBestWater(now, dt, cc4_data, seq_id)

#### Or load results

In [ ]:
with open ('../final_data/optimization_water_474261_str_1', 'rb') as fp:
    W = pickle.load(fp)

### Optimized water

In [ ]:
w_opt = pd.DataFrame(W.x.reshape((30,14)), columns=water_keys)

features_past, features_future, T_soll= getData(now, dt, cc4_data, seq_id)

df_new_f = pd.concat([features_future[feature_future_keys], w_opt], axis=1).reindex(feature_keys, axis=1)
water_opt_unsc = pd.DataFrame(scaler_x.inverse_transform(df_new_f), columns=feature_keys)[water_keys]

In [ ]:
features_past, features_future, T_soll= getData(now, dt, cc4_data, seq_id)

df_new_f = pd.concat([features_future[feature_future_keys], w_opt], axis=1).reindex(feature_keys, axis=1)
water_opt_unsc = pd.DataFrame(scaler_x.inverse_transform(df_new_f), columns=feature_keys)[water_keys]

In [ ]:
fig = plt.figure( figsize=(20, 10))
sns.barplot(data=water_opt_unsc[sorted(water_opt_unsc.columns)], color='dodgerblue')
plt.title('Real Water', fontsize=20, color='white') 

### W0

In [ ]:
w = features_past[water_keys].mean().to_numpy().reshape((1,14))
W0 = pd.DataFrame(np.repeat(w, repeats=dt, axis=0), columns=water_keys)

data = cc4_data[cc4_data['seq_id']==seq_id].reset_index(drop=True)
position = data[data['DATE_TIME']==now].index
features_past_unsc = data.loc[(position[0]-4):position[0]]
features_past_unsc = features_past_unsc[feature_keys]
w_unsc = features_past_unsc[water_keys].mean().to_numpy().reshape((1,14))
W0_unsc = pd.DataFrame(np.repeat(w_unsc, repeats=dt, axis=0), columns=water_keys)

In [ ]:
fig, axes = plt.subplots( figsize=(20, 10),  sharex=True)
sns.barplot(data=W0_unsc[sorted(water_keys)], color='dodgerblue')
plt.title('W0', fontsize=20, color='white') 

In [ ]:
real_water.rename(columns={'WasserZ4FsInLproMin_integr': 'Z4_FS_measured',
                            'WasserZ4LsInLproMin_integr': 'Z4_LS_measured',
                           'WasserZ2bFsInLproMin_integr': 'Z2b_FS_measured',
                            'WasserZ2bLsInLproMin_integr': 'Z2b_LS_measured',
                           'WasserZ3bFsInLproMin_integr': 'Z3b_FS_measured',
                            'WasserZ3bLsInLproMin_integr': 'Z3b_LS_measured',
                           'WasserZ3aFsInLproMin_integr': 'Z3a_FS_measured',
                            'WasserZ3aLsInLproMin_integr': 'Z3a_LS_measured',
                           'WasserZ2aFsInLproMin_integr': 'Z2a_FS_measured',
                            'WasserZ2aLsInLproMin_integr': 'Z2a_LS_measured',
                           'WasserZ5FsInLproMin_integr': 'Z5_FS_measured',
                            'WasserZ5LsInLproMin_integr': 'Z5_LS_measured',
                             'WasserZ1FsInLproMin_integr':'Z1_FS_measured',
                            'WasserZ1LsInLproMin_integr':'Z1_LS_measured',
                           }, inplace=True)

In [ ]:
water_opt_unsc.rename(columns={'WasserZ4FsInLproMin_integr': 'Z4_FS_optimized',
                            'WasserZ4LsInLproMin_integr': 'Z4_LS_optimized',
                           'WasserZ2bFsInLproMin_integr': 'Z2b_FS_optimized',
                            'WasserZ2bLsInLproMin_integr': 'Z2b_LS_optimized',
                           'WasserZ3bFsInLproMin_integr': 'Z3b_FS_optimized',
                            'WasserZ3bLsInLproMin_integr': 'Z3b_LS_optimized',
                           'WasserZ3aFsInLproMin_integr': 'Z3a_FS_optimized',
                            'WasserZ3aLsInLproMin_integr': 'Z3a_LS_optimized',
                           'WasserZ2aFsInLproMin_integr': 'Z2a_FS_optimized',
                            'WasserZ2aLsInLproMin_integr': 'Z2a_LS_optimized',
                           'WasserZ5FsInLproMin_integr': 'Z5_FS_optimized',
                            'WasserZ5LsInLproMin_integr': 'Z5_LS_optimized',
                                 'WasserZ1FsInLproMin_integr':'Z1_FS_optimized',
                            'WasserZ1LsInLproMin_integr':'Z1_LS_optimized',
                           }, inplace=True)

In [ ]:
real_water['type'] = 'measured water'
water_opt_unsc['type'] = 'optimized water'
df = pd.concat([real_water,water_opt_unsc])

df = df.reindex(sorted(df.columns), axis=1)

clrs = [sns.color_palette()[0] if x in real_water.columns else sns.color_palette()[1] for x in df.columns]

In [ ]:
fig = plt.figure( figsize=(12, 10))
sns.barplot(data=df[sorted(df.columns)],  palette=clrs, orient = 'h', ci=None)
plt.title('Real Water', fontsize=20, color='white') 

## Compare prediction with optimized water to original prediction

In [ ]:
y_opt = scaler_y.inverse_transform(predict(w_opt, features_past, features_future))

T_soll_unsc = scaler_y.inverse_transform(T_soll)

y_W0 = scaler_y.inverse_transform(predict(W0, features_past, features_future))

data = cc4_data[cc4_data['seq_id']==seq_id].reset_index()
position = data[data['DATE_TIME']==now].index
real_temp = data.loc[position[0]:position[0]+30][label_keys].to_numpy()

##### RMSE measured Temperature to casting target temperature

In [ ]:
np.sqrt(mean_squared_error( real_temp, T_soll_unsc ))  

##### RMSE predicted Temperature with optimized water to casting target temperature

In [ ]:
np.sqrt(mean_squared_error( y_opt, T_soll_unsc ))  

In [ ]:
fig, (ax1) = plt.subplots(1, 1, figsize=(6, 5))
ax1.plot(T_soll_unsc[:,:1], color='red', label='Target Temperature')
ax1.plot(real_temp[:,:1], color='black', label='Measured Temperature')
ax1.plot(y_opt[:,:1], color='blue', label='Prediction with Optimized Water')
ax1.set_xlabel('Minutes', fontsize=15)
ax1.set_ylabel('Temperatures [°C]', fontsize=15)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
ax1.legend(loc='best')
ax1.legend(prop={'size': 12})
ax1.set_ylim(710, 725)
#ax1.set(yticks=[]) 
ax1.grid()

In [ ]:
fig, (ax1) = plt.subplots(1, 1, figsize=(6, 5))
ax1.plot(T_soll_unsc[:,1:2], color='red', label='Target Temperature')
ax1.plot(real_temp[:,1:2], color='black', label='Measured Temperature')
ax1.plot(y_opt[:,1:2], color='blue', label='Prediction with Optimized Water')
ax1.set_xlabel('Minutes', fontsize=15)
ax1.set_ylabel('Temperatures [°C]', fontsize=15)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
ax1.legend(loc='best')
ax1.legend(prop={'size': 12})
ax1.set_ylim(710, 725)
#ax1.set(yticks=[]) 
ax1.grid()

### Compare water quantities

#### Original water quantities

In [ ]:
fig, (ax1) = plt.subplots(1,1, figsize=(6, 5))
sns.lineplot(data=real_water.reset_index(drop=True),
             y='Z5_LS_measured',x=real_water.reset_index(drop=True).index,
             ax=ax1, label='Z5')
sns.lineplot(data=real_water.reset_index(drop=True),
             y='Z4_LS_measured',x=real_water.reset_index(drop=True).index,
             ax=ax1, label='Z4')
sns.lineplot(data=real_water.reset_index(drop=True),
             y='Z3b_LS_measured',x=real_water.reset_index(drop=True).index,
             ax=ax1, label='Z3b')
sns.lineplot(data=real_water.reset_index(drop=True),
             y='Z3a_LS_measured',x=real_water.reset_index(drop=True).index,
             ax=ax1, label='Z3a')
sns.lineplot(data=real_water.reset_index(drop=True),
             y='Z2b_LS_measured',x=real_water.reset_index(drop=True).index,
             ax=ax1, label='Z2b')
sns.lineplot(data=real_water.reset_index(drop=True),
             y='Z2a_LS_measured',x=real_water.reset_index(drop=True).index,
             ax=ax1, label='Z2a')
sns.lineplot(data=real_water.reset_index(drop=True),
             y='Z1_LS_measured',x=real_water.reset_index(drop=True).index,
             ax=ax1, label='Z1')
ax1.set_ylabel('Cooling Water [l]', fontsize=15)
ax1.xaxis.grid(True)
ax1.set_ylim(0, 400)
ax1.set(yticks=[]) 
ax1.set_xlabel('Minutes', fontsize=15)
ax1.legend(loc='upper left', prop={'size': 13})

In [ ]:
fig.savefig('/home/di40438/bachelorarbeit/data/original_water_ls.png', format='png', dpi=200)

In [ ]:
seq_number = '474261_str_1'
fig, (ax1) = plt.subplots(1,1, figsize=(6, 5))
sns.lineplot(data=real_water.reset_index(drop=True),
             y='Z5_FS_measured',x=real_water.reset_index(drop=True).index,
             ax=ax1, label='Cooling Water Z5')
sns.lineplot(data=real_water.reset_index(drop=True),
             y='Z4_FS_measured',x=real_water.reset_index(drop=True).index,
             ax=ax1, label='Cooling Water Z4')
sns.lineplot(data=real_water.reset_index(drop=True),
             y='Z3b_FS_measured',x=real_water.reset_index(drop=True).index,
             ax=ax1, label='Cooling Water Z3b')
sns.lineplot(data=real_water.reset_index(drop=True),
             y='Z3a_FS_measured',x=real_water.reset_index(drop=True).index,
             ax=ax1, label='Cooling Water Z3a')
sns.lineplot(data=real_water.reset_index(drop=True),
             y='Z2b_FS_measured',x=real_water.reset_index(drop=True).index,
             ax=ax1, label='Cooling Water Z2b')
sns.lineplot(data=real_water.reset_index(drop=True),
             y='Z2a_FS_measured',x=real_water.reset_index(drop=True).index,
             ax=ax1, label='Cooling Water Z2a')
sns.lineplot(data=real_water.reset_index(drop=True),
             y='Z1_FS_measured',x=real_water.reset_index(drop=True).index,
             ax=ax1, label='Cooling Water Z1')
ax1.set_ylabel('Cooling Water [l]', fontsize=15)
ax1.xaxis.grid(True)
ax1.set_ylim(0, 400)
#ax1.set(yticks=[]) 
ax1.set_xlabel('Minutes', fontsize=15)
ax1.legend(loc='upper left', prop={'size': 13})


#### Optimized water quantities

In [ ]:
fig, (ax1) = plt.subplots(1,1, figsize=(6, 5))
sns.lineplot(data=water_opt_unsc.reset_index(drop=True),
             y='Z5_LS_optimized',x=water_opt_unsc.reset_index(drop=True).index,
             ax=ax1, label='Z5')
sns.lineplot(data=water_opt_unsc.reset_index(drop=True),
             y='Z4_LS_optimized',x=water_opt_unsc.reset_index(drop=True).index,
             ax=ax1, label='Z4')
sns.lineplot(data=water_opt_unsc.reset_index(drop=True),
             y='Z3b_LS_optimized',x=water_opt_unsc.reset_index(drop=True).index,
             ax=ax1, label='Z3b')
sns.lineplot(data=water_opt_unsc.reset_index(drop=True),
             y='Z3a_LS_optimized',x=water_opt_unsc.reset_index(drop=True).index,
             ax=ax1, label='Z3a')
sns.lineplot(data=water_opt_unsc.reset_index(drop=True),
             y='Z2b_LS_optimized',x=water_opt_unsc.reset_index(drop=True).index,
             ax=ax1, label='Z2b')
sns.lineplot(data=water_opt_unsc.reset_index(drop=True),
             y='Z2a_LS_optimized',x=water_opt_unsc.reset_index(drop=True).index,
             ax=ax1, label='Z2a')
sns.lineplot(data=water_opt_unsc.reset_index(drop=True),
             y='Z1_LS_optimized',x=water_opt_unsc.reset_index(drop=True).index,
             ax=ax1, label='Z1')
ax1.set_ylabel('Cooling Water [l]', fontsize=15)
ax1.xaxis.grid(True)
ax1.set_ylim(0, 400)
ax1.set(yticks=[]) 
ax1.set_xlabel('Minutes', fontsize=15)
ax1.legend(loc='upper left', prop={'size': 13})

In [ ]:
fig.savefig('/home/di40438/bachelorarbeit/data/optimized_water_ls.png', format='png', dpi=200)

In [ ]:
seq_number = '474261_str_1'
fig, (ax1) = plt.subplots(1,1, figsize=(6, 5))
sns.lineplot(data=water_opt_unsc.reset_index(drop=True),
             y='Z5_FS_optimized',x=water_opt_unsc.reset_index(drop=True).index,
             ax=ax1, label='Cooling Water Z5')
sns.lineplot(data=water_opt_unsc.reset_index(drop=True),
             y='Z4_FS_optimized',x=water_opt_unsc.reset_index(drop=True).index,
             ax=ax1, label='Cooling Water Z4')
sns.lineplot(data=water_opt_unsc.reset_index(drop=True),
             y='Z3b_FS_optimized',x=water_opt_unsc.reset_index(drop=True).index,
             ax=ax1, label='Cooling Water Z3b')
sns.lineplot(data=water_opt_unsc.reset_index(drop=True),
             y='Z3a_FS_optimized',x=water_opt_unsc.reset_index(drop=True).index,
             ax=ax1, label='Cooling Water Z3a')
sns.lineplot(data=water_opt_unsc.reset_index(drop=True),
             y='Z2b_FS_optimized',x=water_opt_unsc.reset_index(drop=True).index,
             ax=ax1, label='Cooling Water Z2b')
sns.lineplot(data=water_opt_unsc.reset_index(drop=True),
             y='Z2a_FS_optimized',x=water_opt_unsc.reset_index(drop=True).index,
             ax=ax1, label='Cooling Water Z2a')
sns.lineplot(data=water_opt_unsc.reset_index(drop=True),
             y='Z1_FS_optimized',x=water_opt_unsc.reset_index(drop=True).index,
             ax=ax1, label='Cooling Water Z1')
ax1.set_ylabel('Cooling Water [l]', fontsize=15)
ax1.xaxis.grid(True)
ax1.set_ylim(0, 400)
#ax1.set(yticks=[]) 
ax1.set_xlabel('Minutes', fontsize=15)
ax1.legend(loc='upper left', prop={'size': 13})
